In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-30957238-4214-7a84-2636-ae83fae7f03d)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
with zipfile.ZipFile("/content/drive/MyDrive/iseg2019/dataISEG_np/train_patch32_stride8.zip", 'r') as zip_ref:
    zip_ref.extractall("data_train")
with zipfile.ZipFile("/content/drive/MyDrive/iseg2019/dataISEG_np/test_patch32_stride8.zip", 'r') as zip_ref:
    zip_ref.extractall("data_test")
# with zipfile.ZipFile("/content/drive/MyDrive/iseg2019/dataISEG_np/pseudo_data.zip", 'r') as zip_ref:
#     zip_ref.extractall("pseudo_data")

In [ ]:
%cd /content/drive/MyDrive/iseg2019/
%run Unet.ipynb
%run data_preprocessing.ipynb

In [ ]:
# !pip install volumentations-3D
# from volumentations import *
# !pip install imjoy scikit-image
#https://github.com/ZFTurbo/volumentations/blob/4e2c73add77d7fdbc5f69d4248c893f6e6aa67dc/volumentations/augmentations/transforms.py#L154
import gc
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.layers import *
import keras.backend as K
from itertools import product
import os
import scipy.io as sio
from scipy import ndimage
import matplotlib.pyplot as plt
import tensorflow_probability as tfp
from tqdm import tqdm
from fastprogress import master_bar, progress_bar
import nibabel as nib
import glob 
!pip install keras-swa
from swa.tfkeras import SWA
from mpl_toolkits import mplot3d
from IPython.display import clear_output
from keras.utils import np_utils
from sklearn.feature_extraction.image import extract_patches as sk_extract_patches
from sklearn.model_selection import train_test_split
from skimage import exposure
clear_output()

##train

In [ ]:
def gradientLoss(y_pred,penalty = "l1"):

    dH = tf.math.abs(y_pred[:,1:,...] - y_pred[:,:-1,...])
    dW = tf.math.abs(y_pred[:,:,1:,...] - y_pred[:,:,:-1,...])
    dD = tf.math.abs(y_pred[...,1:,:] - y_pred[...,:-1,:])
    if penalty == "l2":
        dH = dH * dH
        dW = dW * dW
        dD = dD * dD

    loss =  tf.reduce_sum(dH) +  tf.reduce_sum(dW)+ tf.reduce_sum(dD)
    return loss
def gaussian_kernel(kernel_size: int, mean: float, std: float):
    """Makes 3D gaussian Kernel for convolution."""
    d = tfp.distributions.Normal(mean, std)
    vals = d.prob(tf.range(-kernel_size // 2 + 1.0, kernel_size // 2 + 1.0))
    gauss_kernel = tf.einsum('i,j,k->ijk', vals, vals,vals)
    gauss_kernel = gauss_kernel / tf.reduce_sum(gauss_kernel)
    gauss_kernel = gauss_kernel[...,tf.newaxis, tf.newaxis]
    return gauss_kernel

def levelsetLoss(y_true, y_pred, kernel_size=5, mean_kernel=0, std_kernel=5):
    kernel = gaussian_kernel(kernel_size, mean_kernel, std_kernel)
    kernel = tf.repeat(kernel, y_pred.shape[-1], axis = -1)u
    lossRegion = 0.0
    y_pred_fuzzy = y_pred ** 2
    for ich in range(2):
        target_ = y_true[...,ich:ich+1]   
        pcentroid_1_local = tf.nn.conv3d(target_ * y_pred_fuzzy + 1e-6, kernel, strides=[1,1,1,1,1], padding='SAME')
        pcentroid_2_local =  tf.nn.conv3d(y_pred_fuzzy + 1e-6, kernel, strides=[1,1,1,1,1], padding='SAME')
        pcentroid_local = pcentroid_1_local / pcentroid_2_local
        plevel_local = target_ - pcentroid_local
        loss_local = plevel_local * plevel_local * y_pred_fuzzy

        pcentroid_global = tf.reduce_sum(target_ * y_pred_fuzzy, (1,2,3),keepdims=True)\
                    /tf.reduce_sum(y_pred_fuzzy+1e-6, (1,2,3),keepdims = True)   
        plevel_global = target_ - pcentroid_global
        loss_global = plevel_global * plevel_global * y_pred_fuzzy

        lossRegion += tf.reduce_sum(loss_local)+tf.reduce_sum(loss_global)

    return lossRegion 

def lossGlobalAndLocal(y_true, y_pred, beta = 1e-3):
    lossLevelset = levelsetLoss(y_true, y_pred)
    lossLength = gradientLoss(y_pred)

    return lossLevelset + beta * lossLength

def ActiveContourLoss(y_true, y_pred, smooth=1e-5):   
    axes = (1,2,3,4)
    yTrueOnehot = tf.one_hot(tf.squeeze(y_true,axis=-1), depth = NUM_CLASS)[...,1:]
    #this is Luac:
    y_pred = y_pred[...,1:]

    active = -tf.cast(tf.math.log(1-y_pred+smooth),tf.float32) * (1-yTrueOnehot) - tf.cast(tf.math.log(y_pred+smooth),tf.float32)*yTrueOnehot
    loss = tf.reduce_sum(active, axis = axes) / tf.reduce_sum(yTrueOnehot + y_pred-yTrueOnehot*y_pred+smooth, axis = axes)
    return tf.reduce_mean(loss)

In [ ]:
mcp = ModelCheckpoint("./train_noPseudo/ckpt{val_active_dice_WM:.4f}.h5",mode='max',monitor='val_active_dice_WM',verbose=1,
                      save_best_only=True, save_weights_only=True)
earlystop= EarlyStopping(monitor='val_active_dice_WM',patience=150,mode='max',verbose=1)
rlr = ReduceLROnPlateau(monitor='val_active_dice_WM', factor=0.7, mode='max', patience=20, min_lr=1e-7, verbose=1)
csv_logger = CSVLogger("history.csv",append= True)


In [ ]:
def IsegLoader(typeData = "train", train_path=TRAIN_PATH, test_path= TEST_PATH, 
               shuffle=True, augment=False):
    list_testName = sorted(glob.glob(test_path))
    listName =  glob.glob(train_path) + glob.glob(PSEUDO_PATH) if typeData == "train" else list_testName
    augment =  augment if typeData == "train" else False  # augment data bool
    indexes = np.arange(len(listName))
    if shuffle and typeData == "train":
        np.random.shuffle(indexes)   
    for i in indexes:
        subject = np.load(listName[i])
        volume, mask = subject["image"], subject["mask"]
    ####################### augmentation data ##############################
        # if augment:
        #     volume = gen_augmentation(volume)
        yield {"inputs": volume}, {"active": mask, "levelset": volume}

In [ ]:
train_dataset = tf.data.Dataset.from_generator(IsegLoader, 
                                               output_types=({"inputs": tf.float32}, {"active": tf.uint8, "levelset": tf.float32}))
train_dataset  = train_dataset.batch(16, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(IsegLoader, args=["test"],
                                              output_types=({"inputs": tf.float32}, {"active": tf.uint8, "levelset": tf.float32}))
test_dataset  = test_dataset.batch(64).prefetch(tf.data.AUTOTUNE)

In [ ]:
# S.load_weights("./train_noPseudo/ckpt0.9217.h5")
# for keep_prob in np.linspace(start = 0.96, stop = 0.95, num=2):
#     print("dropout rate is:", keep_prob)
#     new_model = seg_net(keep_prob = keep_prob)
#     new_model.set_weights(S.get_weights())
#     S = new_model
#     S.compile(optimizer=Nadam(learning_rate=7e-4), loss={"active":ActiveContourLoss, 
#                                                             "levelset":lossGlobalAndLocal},\
#             loss_weights={"active": 1.0, "levelset": 1e-8},\
#             metrics={"active": [dice_CSF, dice_GM, dice_WM]})
#     S.fit(train_dataset,epochs = 1, callbacks=[mcp, rlr,earlystop,csv_logger],\
#           validation_data=test_dataset)

In [ ]:
S = seg_net(keep_prob= 0.95)
S.compile(optimizer=Nadam(learning_rate=3e-4), loss={"active":ActiveContourLoss, 
                                                        "levelset":lossGlobalAndLocal},\
        loss_weights={"active": 1.0, "levelset": 1e-8},\
        metrics={"active": [dice_CSF, dice_GM, dice_WM]})


In [ ]:
# swa = SWA(start_epoch=2, 
#           lr_schedule="manual", 
#         #   swa_lr=2e-5,
#         #   swa_lr2=0.001,
#         #   swa_freq=2,
#           batch_size=16,# needed when using batch norm
#           verbose=1)

In [ ]:
S.load_weights("./train_noPseudo/ckpt0.9773.h5")
S.compile(optimizer=Nadam(learning_rate=2e-4), loss={"active":ActiveContourLoss, 
                                                        "levelset":lossGlobalAndLocal},\
        loss_weights={"active": 1.0, "levelset": 1e-8},\
        metrics={"active": [dice_CSF, dice_GM, dice_WM]})

In [ ]:
S.fit(train_dataset,epochs = 1000, callbacks=[mcp, rlr,earlystop,csv_logger], \
      validation_data=test_dataset)

Epoch 1/1000
2233/2233 [==============================] - 3721s 2s/step - loss: 0.0615 - active_loss: 0.0614 - levelset_loss: 10571.8096 - active_dice_CSF: 0.9892 - active_dice_GM: 0.9850 - active_dice_WM: 0.9755 - val_loss: 0.0564 - val_active_loss: 0.0560 - val_levelset_loss: 42108.9883 - val_active_dice_CSF: 0.9909 - val_active_dice_GM: 0.9852 - val_active_dice_WM: 0.9782

Epoch 00001: val_active_dice_WM improved from 0.97650 to 0.97822, saving model to ./train_noPseudo/ckpt0.9782.h5
Epoch 2/1000
 620/2233 [=======>......................] - ETA: 42:11 - loss: 0.0604 - active_loss: 0.0603 - levelset_loss: 10599.2275 - active_dice_CSF: 0.9895 - active_dice_GM: 0.9852 - active_dice_WM: 0.9761

In [ ]:
# S.load_weights("./train_noPseudo/weightSWA.h5")
S.evaluate(test_dataset)

In [ ]:
# if isinstance(layer, DropBlock3D):
#         print(layer.block_size)
# S = load_model("model_U_Unet.h5",
#                custom_objects={"Swish":Swish,"DropBlock3D":DropBlock3D}, compile=False)
# S.compile(optimizer=Nadam(learning_rate=1.25e-4), loss={"active":ActiveContourLoss, 
#                                                         "levelset":levelsetLossCVES},\
#         loss_weights={"active": 1.0, "levelset": 1e-9},\
#         metrics={"active": [dice_CSF, dice_GM, dice_WM]})
# S.evaluate(test_dataset)

##psedo labeling


In [ ]:
def getValidIndex(label_constructed, patchT1, threshold):
    filter = np.max(label_constructed, axis=-1,keepdims=True)
    patch = extract_patches(filter, PATCH_SIZE, EXTRACTION_STEP)
    valid_index = []
    for index in range(patch.shape[0]):
        if (not False in np.unique(patch[index] > threshold)) and np.sum(patchT1[index]) != 0:
            valid_index.append(index)
    return valid_index
# IMG_SUBMIT_PATH = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Validation/"
# LABEL_SUBMIT_PATH = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2017-Test-Prob/"
# for i in range(11,24):
#     print("\rsegment : {}th subject".format(i), end="")
#     imgT1 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T1.hdr").get_fdata()
#     imgT1 = normalize(imgT1)
#     imgT2 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T2.hdr").get_fdata()
#     imgT2 = normalize(imgT2)
#     patchT1 = extract_patches(imgT1, PATCH_SIZE, EXTRACTION_STEP)
#     patchT2 = extract_patches(imgT2, PATCH_SIZE, EXTRACTION_STEP)

#     pred = np.zeros((*patchT1.shape[:-1], NUM_CLASS))
#     pred[...,0] = 1
#     valid_index = np.where(np.sum(patchT1, axis=(1, 2, 3, 4)) != 0)
#     patchT1_valid = tf.convert_to_tensor(patchT1[valid_index], tf.float32)
#     patchT2_valid = tf.convert_to_tensor(patchT2[valid_index], tf.float32)
#     del patchT1, patchT2
#     temp = S.predict({"inputT1": patchT1_valid, 
#                         "inputT2": patchT2_valid}, batch_size = 64)[0]
#     gc.collect()
#     K.clear_session()
#     pred[valid_index] = temp
#     del valid_index, temp, patchT1_valid, patchT2_valid
#     label_constructed = reconstruct_volume_avg(pred, imgT1.shape[:-1], EXTRACTION_STEP)
#     del imgT1, imgT2, pred
#     label_save = nib.Nifti1Image(label_constructed, np.eye(4))
#     nib.save(label_save, LABEL_SUBMIT_PATH+"subject-"+str(i)+"-label.nii.gz")    
#     del label_constructed, label_save


In [ ]:
# count_train = 1
# PSEUDO_PATH = "/content/pseudo_data/"
# GET_LABEL_PATH  = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2017-Test-Prob/"
# GET_IMAGE_PATH  = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2017-Testing/"
# for i in range(11, 24):
#     label_constructed = nib.load(GET_LABEL_PATH+"subject-"+str(i)+"-label.nii.gz").get_fdata()
#     imgT1 = nib.load(GET_IMAGE_PATH+"subject-"+str(i)+"-T1.hdr").get_fdata()
#     imgT1 = normalize(imgT1)
#     patchT1 = extract_patches(imgT1, PATCH_SIZE, EXTRACTION_STEP)
#     valid_index = getValidIndex(label_constructed, patchT1, 0.5)

#     x_t1 = patchT1[valid_index]
#     del patchT1, imgT1
#     output_standard = np.expand_dims(np.argmax(label_constructed, axis=-1), axis=-1)
#     label_patch = extract_patches(output_standard,PATCH_SIZE,EXTRACTION_STEP)
#     label = label_patch[valid_index]
#     del label_patch, output_standard, label_constructed
#     imgT2 = nib.load(GET_IMAGE_PATH+"subject-"+str(i)+"-T2.hdr").get_fdata()
#     imgT2 = normalize(imgT2)
#     patchT2 = extract_patches(imgT2, PATCH_SIZE, EXTRACTION_STEP)
#     x_t2 = patchT2[valid_index]
#     del patchT2, imgT2 

#     for j in range(len(valid_index)):
#         print("\rsave ",count_train,"th volume for train", end="")             
#         imgT1 = nib.Nifti1Image( x_t1[j], np.eye(4))
#         nib.save(imgT1, PSEUDO_PATH+ 'T1_train_'+str(count_train)+'.nii.gz')  
#         imgT2 = nib.Nifti1Image( x_t2[j], np.eye(4))
#         nib.save(imgT2, PSEUDO_PATH+ 'T2_train_'+str(count_train)+'.nii.gz')  
#         img_mask = nib.Nifti1Image(label[j], np.eye(4))
#         nib.save(img_mask, PSEUDO_PATH+'mask_train_'+str(count_train)+'.nii.gz')    
#         count_train += 1 
# import shutil
# shutil.make_archive("/content/drive/MyDrive/iseg2019/pseudo_data", "zip","/content/pseudo_data") 

##submit

In [ ]:
S.inputs

In [ ]:
def convertLable2Submit(label):
    class_mapper_inv = {1 : 10, 2 : 150, 3 : 250}
    for key, value in class_mapper_inv.items():
        label = np.where(label==key, value, label)
    return label.astype(np.uint8)

In [ ]:
S = load_model("model_U_Unet.h5",
               custom_objects={"Swish":Swish,"DropBlock3D":DropBlock3D}, compile=False)

In [ ]:
# dice_csf = []
# dice_gm = []
# dice_wm = []
# for j in range(1,3):
#     imgT1 = nib.load("./dataISEG/iSeg-2019-Training/subject-"+str(j)+"-T1.hdr").get_fdata()
#     imgT1 = min_max_preprocess(imgT1)
#     imgT2 = nib.load("./dataISEG/iSeg-2019-Training/subject-"+str(j)+"-T2.hdr").get_fdata()
#     imgT2 = min_max_preprocess(imgT2)
#     label = nib.load("./dataISEG/iSeg-2019-Training/subject-"+str(j)+"-label.hdr").get_fdata()
    # del imgT1, imgT2
#     inputs = np.concatenate([imgT1, imgT2], axis= -1)
#     input_patches = extract_patches(inputs, PATCH_SIZE, EXTRACTION_STEP)
#     input_patches = tf.convert_to_tensor(input_patches, tf.float32)

#     pred = S.predict(input_patches, batch_size = 64)[0]
#     del input_patches
#     gc.collect()
#     K.clear_session()
#     label_constructed = reconstruct_volume_avg(pred, inputs.shape[:-1], EXTRACTION_STEP)
#     del pred
#     output_standard = tf.expand_dims(tf.argmax(label_constructed,axis=-1), axis=-1)
#     del label_constructed
#     for i in range(1,4):
#         output_CSF = np.where(output_standard == i, 1.0, 0.0)
#         label_CSF = np.where(label == i, 1.0, 0.0)

#         intersection = K.sum(label_CSF * output_CSF, axis=[0,1,2,3])
#         union = K.sum(label_CSF, axis=[0,1,2,3]) + K.sum(output_CSF, axis=[0,1,2,3])
#         dice = ((2. * intersection + 1) / (union + 1))
#         if i==1:
#             dice_csf.append(dice)
#         elif i==2:
#             dice_gm.append(dice)
#         else : 
#             dice_wm.append(dice)
#         print(f"subject {j}: ", dice)
#     del intersection, union, dice, label, output_standard     
# print(np.mean(dice_csf), np.mean(dice_gm),np.mean(dice_wm))                    

In [ ]:
IMG_SUBMIT_PATH = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Validation/"
LABEL_SUBMIT_PATH = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Validation-Label/"
for i in range(11,24):
    print("\rsegment : {}th subject".format(i), end="")
    affine = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T1.hdr").affine
    imgT1 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T1.hdr").get_fdata()
    imgT1 = min_max_preprocess(imgT1)
    imgT2 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T2.hdr").get_fdata()
    imgT2 = min_max_preprocess(imgT2)

    inputs = np.concatenate([imgT1, imgT2], axis= -1)
    del imgT1, imgT2
    input_patches = extract_patches(inputs, PATCH_SIZE, EXTRACTION_STEP)
    input_patches = tf.convert_to_tensor(input_patches, tf.float32)

    pred = S.predict(input_patches, batch_size = 64)[0]
    del input_patches
    gc.collect()
    K.clear_session()
    label_constructed = reconstruct_volume_avg(pred, inputs.shape[:-1], EXTRACTION_STEP)
    del pred
    output_standard = tf.expand_dims(tf.argmax(label_constructed,axis=-1), axis=-1)
    del label_constructed

    label_save = nib.Nifti1Image(output_standard,affine)
    nib.nifti1.save(label_save, LABEL_SUBMIT_PATH+"subject-"+str(i)+"-label.hdr")
    del output_standard

In [ ]:
IMG_SUBMIT_PATH = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Validation/"
LABEL_SUBMIT_PATH = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Validation-Label/"
for i in range(11,24):
    print("\rsegment : {}th subject".format(i), end="")
    imgT1 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T1.hdr").get_fdata()
    imgT1 = normalize(imgT1)
    imgT2 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T2.hdr").get_fdata()
    imgT2 = normalize(imgT2)
    patchT1 = extract_patches(imgT1, PATCH_SIZE, EXTRACTION_STEP)
    patchT2 = extract_patches(imgT2, PATCH_SIZE, EXTRACTION_STEP)

    pred = np.zeros((*patchT1.shape[:-1], NUM_CLASS))
    pred[...,0] = 1
    valid_index = np.where(np.sum(patchT1, axis=(1, 2, 3, 4)) != 0)
    patchT1_valid = tf.convert_to_tensor(patchT1[valid_index], tf.float32)
    patchT2_valid = tf.convert_to_tensor(patchT2[valid_index], tf.float32)
    del patchT1, patchT2
    temp = S.predict({"inputT1": patchT1_valid, 
                        "inputT2": patchT2_valid}, batch_size = 32)[0]
    gc.collect()
    K.clear_session()
    pred[valid_index] = temp
    del valid_index, temp, patchT1_valid, patchT2_valid
    label_constructed = reconstruct_volume_avg(pred, imgT1.shape[:-1], EXTRACTION_STEP)
    output_standard = np.expand_dims(np.argmax(label_constructed, axis=-1), axis=-1).astype(np.uint8)
    del imgT1, imgT2, pred, label_constructed

    label_save = nib.Nifti1Image(output_standard, np.eye(4))
    nib.nifti1.save(label_save, LABEL_SUBMIT_PATH+"subject-"+str(i)+"-label.hdr")
    del output_standard

In [ ]:
IMG_SUBMIT_PATH = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Testing/"
LABEL_SUBMIT_PATH = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Test-label/"
for i in range(24,40):
    print("\rsegment : {}th subject".format(i), end="")
    imgT1 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T1.hdr").get_fdata()
    imgT1 = normalize(imgT1)
    imgT2 = nib.load(IMG_SUBMIT_PATH+"subject-"+str(i)+"-T2.hdr").get_fdata()
    imgT2 = normalize(imgT2)
    patchT1 = extract_patches(imgT1, PATCH_SIZE, EXTRACTION_STEP)
    patchT2 = extract_patches(imgT2, PATCH_SIZE, EXTRACTION_STEP)
    pred = np.zeros((*patchT1.shape[:-1], NUM_CLASS))
    valid_index = np.where(np.sum(patchT1, axis=(1, 2, 3, 4)) != 0)
    pred[valid_index] = S.predict({"inputT1": patchT1[valid_index],"inputT2": patchT2[valid_index]})[0]
    label_constructed = reconstruct_volume_avg(pred, imgT1.shape[:-1], EXTRACTION_STEP)
    output_standard = np.expand_dims(np.argmax(label_constructed, axis=-1), axis=-1).astype(np.uint8)

    label_save = nib.Nifti1Image(output_standard, np.eye(4))
    nib.nifti1.save(label_save, LABEL_SUBMIT_PATH+"subject-"+str(i)+"-label.hdr")
    del imgT1, imgT2, patchT1, patchT2, pred, label_constructed, output_standard,label_save,valid_index

In [ ]:
j = 1
imgT1 = nib.load("/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Training/subject-"+str(j)+"-T1.hdr").get_fdata()
imgT1 = normalize(imgT1)
imgT2 = nib.load("/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Training/subject-"+str(j)+"-T2.hdr").get_fdata()
imgT2 = normalize(imgT2)
label = nib.load("/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Training/subject-"+str(j)+"-label.hdr").get_fdata()

In [ ]:
imgT1 = nib.load("/content/pseudo_data/T1_train_2000.nii.gz").get_fdata()
imgT2 = nib.load("/content/pseudo_data/T2_train_2000.nii.gz").get_fdata()
label = nib.load("/content/pseudo_data/mask_train_2001.nii.gz").get_fdata()

In [ ]:
np.unique(label)

In [ ]:
for i in range(32):
    plt.figure(i+1, figsize=(24,24))
    plt.subplot(131),plt.imshow(imgT1[:,:,i,0]),plt.title('T1')
    plt.subplot(132),plt.imshow(imgT2[:,:,i,0]),plt.title('T2')
    plt.subplot(133),plt.imshow(label[:,:,i,0]),plt.title('label')

In [ ]:
a

In [ ]:
for i in range(128, 150):
    plt.figure(i+1, figsize=(24,24))
    plt.subplot(131),plt.imshow(a[0,35:110,80:155,i,0]),plt.title('T1')
    # plt.subplot(131),plt.imshow(imgT1[35:110,80:155,i,0]),plt.title('T1')
    plt.subplot(132),plt.imshow(x_trainT1[0,35:110,80:155,i,0]),plt.title('T2')
    # plt.subplot(133),plt.imshow(label[35:110,80:155,i,0]),plt.title('label')

In [ ]:
path1 = sorted(glob.glob("/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Validation/*.hdr"))
path2 = sorted(glob.glob("/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2017-Testing/*.hdr"))
for i in range(len(path1)):
    ar1 = nib.load(path1[i]).get_fdata()
    ar2 = nib.load(path2[i]).get_fdata()
    k =np.linalg.norm(ar1-ar2)
    if k>0:
        print(k)

In [ ]:
# path = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Validation/*.hdr"
# path = "/content/drive/MyDrive/iseg2019/dataISEG/iSeg-2019-Training/*.hdr"

# for i in glob.glob(path):
#     ar = nib.load(i).get_fdata()
#     aug = CenterCrop(CROP_SIZE).apply(ar)
#     aug = aug[:,16:,16:,:]
#     # k = np.sum(aug[:,:16,:,:]) 
#     k = np.sum(ar)- np.sum(aug)
#     if k > 0:
#         print(i+f" with {k}")